In [2]:
import torch
from torch import nn

In [3]:
ll = nn.Linear(1, 12, bias=False)

In [15]:
tok = ll(torch.tensor([1.]).unsqueeze(0))
tok = tok.unsqueeze(0)
tok = tok.repeat(5, 1, 1)
tok.shape

torch.Size([5, 1, 12])

In [16]:
x = torch.rand([5, 10, 12])
x.shape

torch.Size([5, 10, 12])

In [18]:
y = torch.concatenate([tok, x], 1)
y.shape

torch.Size([5, 11, 12])

In [19]:
ones = torch.ones([3,3,3], dtype=torch.float32)